In [74]:
import os, sys, re, warnings, logging, pickle, bz2
from os.path import join
from glob import glob
import numpy as np
from scipy.signal import find_peaks
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
sys.path.append('..')
import trilabtracker as tt

from importlib import reload
for m in tt.__all__:
    eval(f'reload(tt.{m})')
reload(tt)

plt.rcParams['figure.figsize'] = 6,4
dpi = 150

R_cm         = 55.5

os.getcwd()

'/media/disk2/adult_schooling/trilab-tracker/paper-figures'

## Prepare trial data.

Prepare a dictionary of trials to analyze with basic info for each (path to trial file, population, age, number of individuals, etc).

In [81]:
# Load list of trial names to use in the analysis.
valid_trials = open('../settings/adult-trials-to-analyze.txt').readlines()
valid_trials = [ fn.strip() for fn in valid_trials ]

# Extract trial metadata from the trial's filename.
def parse_trial_file(trial_file, etho=False):
#     if etho:
#         trial_dir  = None
#         trial_name = os.path.basename(trial_file)
#         trial_name = trial_name.split('-')[1]
#     else:
    trial_dir  = os.path.dirname(trial_file)
    trial_name = os.path.basename(trial_dir)
    _,pop,n    = trial_name.lower().split('_')[:3]
    n_ind      = int(n[1:])
    if 'dark' in trial_name.lower():
        pop = pop+'-dark'
        # Skip dark trials until I implement one-sided background subtraction.
        return None
    
    if not trial_name in valid_trials:
        return None
    trial      = { k:v for k,v in locals().items() if k in ['trial_file', 
                              'trial_dir', 'trial_name', 'pop', 'n_ind'] }
    trial['R_cm'] = R_cm
    return trial

# trilabtracker only for now
def load_trial(trial_file, **args):
    trial = parse_trial_file(trial_file)
    return tt.preprocess_trial(trial, **args)
    ''' !!!! '''
    # [?] Replace orientations (data[:,:,2]) with displacement-based ones.
    # [!] Handle overlaps.
    ''' !!!! '''    

# Select a set of trials to analyze.
trial_files = sorted(glob('../tracking_output/*/trial.pik'))
# print(trial_files)

# Count trials of each type.
trials = [ parse_trial_file(f) for f in trial_files ]
trials = pd.DataFrame(trials, index=trial_files)
grouped_trials = trials.groupby(['pop','n_ind'])
count  = pd.DataFrame(grouped_trials['trial_dir'].count().rename('count'))
count = count.unstack(1)
count.columns = count.columns.droplevel()
count[pd.isna(count)] = 0
count = count.astype(int)
display(count)

def matching_trials(pop=None, n_ind=None, df=trials):
    I = pd.Series(data=True, index=df.index)
    if not pop is None:
        I = I & (df['pop']==pop)
    if not n_ind is None:
        I = I & (df['n_ind']==n_ind)
    return df[I].index.tolist()

n_ind,1,2,5,10
pop,,,,
mo,4,4,4,4
pa,20,9,10,4
sf,20,10,12,10
ti,4,4,4,3


In [82]:
cut_ranges0 = { 'v':[0,100], 'v_ang':[-30,30], 't':[600,1800] }
cut_ranges1 = { 'v':[1,100], 'v_ang':[-30,30], 't':[600,1800] }

# Figure 1

In [ ]:
# fig_data = dict(
#     bins_dWall    = R_cm - np.sqrt(np.linspace(R_cm**2,0,20)), 
# #     bins_v        = np.linspace(*cut_ranges['v'],20), 
# #     bins_vAng     = np.linspace(*cut_ranges['v_ang'],20), 
# #     bins_pairDist = np.linspace(0,2*R_cm,60), 
# #     bins_pairAng  = np.linspace(0,np.pi,30), 
#     )
# globals().update(fig_data)

In [90]:
fig1        = { k:defaultdict(list) for k in ['density', 'nematic', 'median'] }
bins        = R_cm - np.sqrt(np.linspace(R_cm**2,0,20))
bin_centers = (bins[1:]+bins[:-1])
for trial_file in tqdm(matching_trials(n_ind=1)):
    trial = load_trial(trial_file, load_timestamps=False, cut_ranges=cut_ranges0)
    globals().update(trial)
    # Area density and nematic OP vs distance to the wall.
    vals       = d_wall.flatten()
    median     = np.nanmedian(vals)
    density,_  = np.histogram(vals, bins=bins)
    area       = np.pi*(R_cm-bins[:-1])**2 - np.pi*(R_cm-bins[1:])**2
    density    = density/np.sum(density)/area
    nematic    = []
    for i in range(len(bins)-1):
        I = (vals>=bins[i])&(vals<bins[i+1])
        thetaP = np.arctan2(pos[I,0,1],pos[I,0,0]) # position angle (polar angle)
        thetaV = np.arctan2(vel[I,0,1],vel[I,0,0]) # velocity angle
        thetaW = thetaV-thetaP # velocity angle with respect to closest wall
        thetaW = thetaW - 2*np.pi*np.rint(thetaW/(2*np.pi)) # between -pi and pi
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            nematic.append(np.nanmean(np.cos(2*thetaW)))
    fig1['density'][pop].append(density)
    fig1['nematic'][pop].append(nematic)
    fig1['median'][pop].append(median)
#     break
for k1,v1 in fig1.items():
    for k2,v2 in v1.items():
        fig1[k1][k2] = np.array(v2)
fig1['bin_centers'] = bin_centers

# pickle.dump( dict(fig1=fig1), open('figure-data.pik','wb') )

100%|██████████| 48/48 [00:01<00:00, 40.44it/s]


In [84]:
pickle.dump( dict(fig1=fig1), open('figure-data.pik','wb') )

In [99]:
fig2        = { k:defaultdict(list) for k in ['speed_distribution', 'inactive_fraction'] }
bins        = np.linspace(0,100,101)
bin_centers = (bins[1:]+bins[:-1])/2
v_inactive  = 1
for trial_file in tqdm(trials.index):
    trial = load_trial(trial_file, load_timestamps=False, cut_ranges=cut_ranges0)
    globals().update(trial)
    if pop=='sf':
        h,_  = np.histogram(v.flatten(), bins=bins, density=True)
        fig2['speed_distribution'][pop,n_ind].append(h)
    f = np.count_nonzero(v>v_inactive)/np.count_nonzero(np.isfinite(v))
    fig2['inactive_fraction'][pop,n_ind].append(f)
for k,v in fig2['speed_distribution'].items():
    fig2['speed_distribution'][k] = np.array(v)
fig2['speed_distribution']['bin_centers'] = bin_centers

100%|██████████| 126/126 [00:06<00:00, 19.79it/s]


In [100]:
pickle.dump( dict(fig1=fig1, fig2=fig2), open('figure-data.pik','wb') )

In [ ]:

# ### Analyze trials.

# traj_data = {}
# stat_data = dict(
#     bins_dWall    = np.linspace(0,R_cm,100), 
#     bins_v        = np.linspace(*cut_ranges['v'],100), 
#     bins_vAng     = np.linspace(*cut_ranges['v_ang'],100), 
#     bins_pairDist = np.linspace(0,2*R_cm,60), 
#     bins_pairAng  = np.linspace(0,np.pi,30), 
#     )
# globals().update(stat_data)
# results = {}

# for trial_file in tqdm(trials.index):
    
#     trial = load_trial(trial_file, load_timestamps=False, cut_ranges=cut_ranges)
#     globals().update(trial)
    
#     # [?] Replace orientations (data[:,:,2]) with displacement-based ones.
    
#     # Save trajectories.
#     traj_data[trial_name] = pos[:,:,:2]
    
#     # [!] Handle overlaps.
    
#     # Distribution of distance to the wall.
#     bins, vals = bins_dWall, d_wall.flatten()
#     hist_dWall = np.histogram(vals[~np.isnan(vals)], bins=bins)
    
#     # Speed distribution.
#     bins, vals = bins_v, v.flatten()
#     hist_v = np.histogram(vals[~np.isnan(vals)], bins=bins)
    
#     # Angular speed distribution.
#     bins, vals = bins_vAng, vel[:,:,2].flatten()
#     hist_vAng = np.histogram(vals[~np.isnan(vals)], bins=bins)
    
#     # Joint distribution of pair distance and pair angle,
#     if n_ind>1:
#         bins_d  = bins_pairDist
#         bins_a  = bins_pairAng
#         J1,J2 = np.triu_indices(n_ind,1)
#         d     = np.hypot(pos[:,J1,0]-pos[:,J2,0],pos[:,J1,1]-pos[:,J2,1]).flatten()
#         a     = (pos[:,J1,2]-pos[:,J2,2]).flatten()
#         a     = a - 2*np.pi*np.rint(a/(2*np.pi))
#         I     = np.logical_not(np.logical_or(np.isnan(d),np.isnan(a)))
#         d     = d[I]
#         a     = np.absolute(a[I])
#         # 2D histogram of pairwise distance and angle.
#         hist_distAng = np.histogram2d(d, a, bins=(bins_d,bins_a), density=True)
#         # Pairwise polar alignment parameter vs pair distance.
#         K = np.digitize(d,bins_d)
#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore", category=RuntimeWarning)
#             p = np.array([ np.nanmean(np.cos(a[K==i])) for i in range(len(bins_d)+1) ])
#         polar = p[1:-1],bins_d
#     else:
#         hist_distAng,polar = None,None

#     # Save output.
#     results[trial_file] = { k:v for k,v in locals().items() if k in 
#                                [ 'valid_fraction', 'hist_area', 'hist_aspect', 'hist_dWall', 
#                                  'hist_v', 'hist_vAng', 'hist_distAng', 'polar' ] }
# #     break

# stat_data['results'] = results
# pickle.dump(stat_data, open(join(cut_dir,'stats.pik'), 'wb'))
# pickle.dump(traj_data, open(join(cut_dir,'trajectories.pik'), 'wb'))